# Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=561

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [3]:
def get_image_and_json_paths(src_path):
    image_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.jpg'), recursive=True))
    json_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.json'), recursive=True))

    return image_paths, json_paths

def extract_metadata_and_locations_from_json(json_paths):
    meta_data, polygon_data, box_data = [], [], []
    for json_path in tqdm(json_paths, desc='Loading JSON', unit=' file'):
        try:
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.loads(re.sub(r'[\x00-\x1F\x7F-\x9F]', '', file.read()))
                labeling_info, metadata = json_data['labelingInfo'], json_data.get('metaData', None)

                filtered_metadata = {
                    'breed': metadata.get('breed', None),
                    'age': metadata.get('age', None),
                    'gender': metadata.get('gender', None),
                    'region': metadata.get('region', None),
                    'lesions': metadata.get('lesions', None)
                }

                meta_data.append(filtered_metadata)

                for entry in labeling_info:
                    if 'polygon' in entry:
                        polygon_data.extend(entry['polygon'].get('location', []))
                    if 'box' in entry:
                        box_data.extend(entry['box'].get('location', []))

        except Exception as e:
            print(json_path)
            print(e)

    return meta_data, polygon_data, box_data

def combine_images_and_masks(image_list, mask_list):
    combined_images = []

    for i in tqdm(range(len(image_list)), desc='Combining Images and Masks'):
        image = image_list[i]
        mask = mask_list[i]

        # Create an empty mask channel with the same shape as the image
        mask_channel = np.zeros_like(image[:, :, 0], dtype=np.uint8)

        # Set the mask channel to the mask values
        mask_channel[mask == 1] = 255

        # Stack the mask channel with the image
        combined_image = np.dstack((image, mask_channel))

        combined_images.append(combined_image)

    return combined_images

def one_hot_encode_labels(labels):
    label_binarizer = LabelBinarizer()
    encoded_labels = label_binarizer.fit_transform(labels)
    return np.array(encoded_labels), label_binarizer

def create_polygon_binary_masks(image_list, polygon_data_list):
    binary_masks = []

    for image, polygon_data in tqdm(zip(image_list, polygon_data_list), desc='Generating Binary Masks', total=len(image_list)):
        mask = np.zeros(image.shape[:2], dtype=np.uint8)

        for poly_coords in polygon_data:
            if poly_coords:
                poly_points = []
                i = 1
                while f'x{i}' in poly_coords and f'y{i}' in poly_coords:
                    x = poly_coords[f'x{i}']
                    y = poly_coords[f'y{i}']
                    poly_points.append([x, y])
                    i += 1
                if len(poly_points) > 0:
                    poly_points = np.array(poly_points, dtype=np.int32)
                    cv2.fillPoly(mask, [poly_points], 255)
            print(poly_coords)

        binary_masks.append(mask)

    return binary_masks

def visualize_masks(image_data, mask_data, selected_indices, figsize=(10, 5)):
    for idx in selected_indices:
        image = image_data[idx]
        mask_map = mask_data[idx]

        plt.figure(figsize=figsize)
        plt.subplot(1, mask_map.shape[2] + 1, 1)
        plt.imshow(image)
        plt.title('Original Image')

        for ch in range(mask_map.shape[2]):
            plt.subplot(1, mask_map.shape[2] + 1, ch + 2)
            plt.imshow(mask_map[:, :, ch], cmap='gray')
            plt.title(f'Channel {ch}')

        plt.tight_layout()
        plt.show()

# camera cat

In [4]:
src_path = '/content/drive/Shareddrives/반려묘/일반카메라'

In [5]:
image_paths, json_paths = get_image_and_json_paths(src_path)

In [6]:
meta_data, polygon_data, box_data = extract_metadata_and_locations_from_json(json_paths)

Loading JSON: 100%|██████████| 5500/5500 [00:04<00:00, 1206.40 file/s]


In [7]:
metadata_df = pd.DataFrame(meta_data)
metadata_df['image_paths'] = image_paths

In [8]:
metadata_df

,breed,age,gender,region,lesions,image_paths
0,노르웨이숲,10,M,H,A7,/content/drive/Shareddrives/반려묘/일반카메라/무증상...
1,노르웨이숲,10,M,B,A7,/content/drive/Shareddrives/반려묘/일반카메라/무증상...
2,노르웨이숲,4,M,B,A7,/content/drive/Shareddrives/반려묘/일반카메라/무증상...
3,노르웨이숲,10,F,B,A7,/content/drive/Shareddrives/반려묘/일반카메라/무증상...
4,노르웨이숲,10,F,B,A7,/content/drive/Shareddrives/반려묘/일반카메라/무증상...
...,...,...,...,...,...,...
5495,페르시안,2,F,L,A6,/content/drive/Shareddrives/반려묘/일반카메라/유증상...
5496,코리안숏헤어,8,F,H,A6,/content/drive/Shareddrives/반려묘/일반카메라/유증상...
5497,코리안숏헤어,5,M,H,A6,/content/drive/Shareddrives/반려묘/일반카메라/유증상...
5498,코리안숏헤어,2,F,H,A6,/content/drive/Shareddrives/반려묘/일반카메라/유증상...


In [9]:
datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

In [10]:
train_df, temp_df = train_test_split(metadata_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [11]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_paths',
    y_col='lesions',
    class_mode='categorical',
    target_size=(96, 96),
    batch_size=32
)

Found 4400 validated image filenames belonging to 4 classes.


In [12]:
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_paths',
    y_col='lesions',
    class_mode='categorical',
    target_size=(96, 96),
    batch_size=32
)

Found 550 validated image filenames belonging to 4 classes.


In [13]:
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_paths',
    y_col='lesions',
    class_mode='categorical',
    target_size=(96, 96),
    batch_size=32,
    shuffle=False
)

Found 550 validated image filenames belonging to 4 classes.


In [14]:
model = models.Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(96,96,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(units=4, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history1 = model.fit(
    train_generator,
    epochs=150,
    batch_size=32,
    validation_data=val_generator
)

Epoch 1/150
138/138 [==============================] - 78s 510ms/step - loss: 1.6290 - accuracy: 0.4693 - val_loss: 1.4930 - val_accuracy: 0.5618
Epoch 2/150
138/138 [==============================] - 71s 516ms/step - loss: 1.2807 - accuracy: 0.5418 - val_loss: 1.6258 - val_accuracy: 0.2782
Epoch 3/150
138/138 [==============================] - 71s 512ms/step - loss: 1.2126 - accuracy: 0.5564 - val_loss: 1.3351 - val_accuracy: 0.6055
Epoch 4/150
138/138 [==============================] - 70s 508ms/step - loss: 1.1764 - accuracy: 0.5580 - val_loss: 1.3600 - val_accuracy: 0.4927
Epoch 5/150
138/138 [==============================] - 70s 511ms/step - loss: 1.0821 - accuracy: 0.5841 - val_loss: 0.9237 - val_accuracy: 0.6309
Epoch 6/150
138/138 [==============================] - 70s 508ms/step - loss: 1.0631 - accuracy: 0.6025 - val_loss: 0.9204 - val_accuracy: 0.6509
Epoch 7/150
138/138 [==============================] - 70s 505ms/step - loss: 1.0703 - accuracy: 0.5995 - val_loss: 0.9844 -

In [ ]:
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = test_generator.classes

print(classification_report(y_true_classes, y_pred_classes))

18/18 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.43      0.15      0.22        41
           1       0.82      0.63      0.71        99
           2       0.70      0.44      0.54       101
           3       0.72      0.93      0.81       309

    accuracy                           0.72       550
   macro avg       0.67      0.53      0.57       550
weighted avg       0.71      0.72      0.70       550



In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('72.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
combined_df = pd.concat([pd.DataFrame(resized_images), metadata_df.drop(columns='lesions')], axis=1)

ValueError: ignored

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(combined_df, encoded_labels, test_size=0.2, random_state=42)
test_images, val_images, test_labels, val_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [ ]:
history2 = model.fit(train_images, train_labels, epochs=150, batch_size=32, validation_data=(val_images, val_labels))

In [ ]:
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
polygon_binary_masks = create_polygon_binary_masks(image_data[:5], polygon_data[:5])

In [ ]:
def visualize_binary_masks(mask_list, selected_indices, figsize=(10, 5)):
    for idx in selected_indices:
        mask = mask_list[idx]

        plt.figure(figsize=figsize)
        plt.imshow(mask, cmap='gray')
        plt.title("Binary Mask")

        plt.tight_layout()
        plt.show()

# 예시 데이터
selected_indices = [0, 5, 10]  # 선택한 이미지 인덱스 리스트

# 시각화
visualize_binary_masks(polygon_binary_masks, selected_indices)

In [ ]:
polygon_data[:2]

In [ ]:
box_data[:2]

In [ ]:
polygon_binary_masks = [mask_map[..., 0] for mask_map in masks_maps]

In [ ]:
polygon_binary_masks = combine_images_and_masks(image_data, polygon_binary_masks)

In [ ]:
visualize_masked_images(image_data, polygon_binary_masks, selected_indices=[0, 5, 10])

In [ ]:
resized_images = resize_images(image_data, 96, 96)
normalized_images = normalize_images(resized_images)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(resized_images, encoded_labels, test_size=0.3, random_state=42)
test_images, val_images, test_labels, val_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [ ]:
history = model.fit(train_images, train_labels, epochs=150, batch_size=32, validation_data=(val_images, val_labels))

In [ ]:
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
polygon_segmentation_maps = [mask_map[..., 1] for mask_map in masks_maps]

In [ ]:
polygon_segmentation_maps = combine_images_and_masks(image_data, polygon_segmentation_maps)

In [ ]:
visualize_masked_images(image_data, polygon_segmentation_maps, selected_indices=[0, 5, 10])

In [ ]:
box_binary_masks = [mask_map[..., 2] for mask_map in masks_maps]

In [ ]:
box_binary_masks = combine_images_and_masks(image_data, box_binary_masks)

In [ ]:
visualize_masked_images(image_data, box_binary_masks, selected_indices=[0, 5, 10])

In [ ]:
box_segmentation_maps = [mask_map[..., 3] for mask_map in masks_maps]

In [ ]:
box_segmentation_maps = combine_images_and_masks(image_data, box_segmentation_maps)

In [ ]:
visualize_masked_images(image_data, box_segmentation_maps, selected_indices=[0, 5, 10])

In [ ]:
original_size_mask_maps = resize_images(masks_maps, 96, 96)

In [ ]:
resized_mask_maps = create_mask_maps(resize_images(image_data, 96, 96), polygon_data, box_data)

In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('your_model.tflite', 'wb') as f:
    f.write(tflite_model)

# microscope cat

In [ ]:
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Concatenate, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mean_squared_error

def inception_module(input_layer, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), activation='relu')(input_layer)
    conv3x3_reduce = Conv2D(filters[1], (1, 1), activation='relu')(input_layer)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)
    conv5x5_reduce = Conv2D(filters[3], (1, 1), activation='relu')(input_layer)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_layer)
    maxpool_conv = Conv2D(filters[5], (1, 1), activation='relu')(maxpool)
    inception_output = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool_conv])
    return inception_output

# Input
input_shape = (128, 128, 3)
input_layer = Input(shape=input_shape)

# Inception block
inception_output = inception_module(input_layer, filters=[64, 128, 192, 32, 96, 64])
inception_output = inception_module(inception_output, filters=[64, 128, 192, 32, 96, 64])
# Add more inception modules if needed

# Primary Capsule layer
primary_capsules = Conv2D(32, (1, 1), activation='relu')(inception_output)

# Higher Capsule layers
# (Add imperative routing mechanism layers here)

# PReLU activation for routing
higher_capsules_prelu = PReLU()(higher_capsules)

# Flatten and Fully Connected layers
capsule_flatten = Flatten()(higher_capsules_prelu)  # Flatten higher capsules
output_layer = Dense(2, activation='softmax')(capsule_flatten)  # Two capsules: parasitized and uninfected

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with Adam optimizer and custom loss function
optimizer = Adam(learning_rate=0.007, beta_1=0.8)
loss_fn = custom_loss_function # Define the custom loss function as described in the paper
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()